# Домашнее задание 3 [10 баллов] 
# До 30.04.18 23:59

Задание выполняется в группе (1-4 человека). В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.

Не все части обязательны для выполнения, однако вы можете быть дополнительно оштрафованы за небрежное за выполнение одной или двух частей вместо четырех.

При возниконовении проблем с выполнением задания обращайтесь с вопросами к преподавателю. Поэтому настоятельно рекомендуется выполнять задание заранее, оставив запас времени на всевозможные технические проблемы. Если вы начали читать условие в последний вечер и не успели из-за проблем с установкой какой-либо библиотеки — это ваши проблемы.


Результат выполнения задания — это отчёт в формате html на основе Jupyter Notebook. Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию
* Подробный пошаговый рассказ о проделанной работе
* Аккуратно оформленные результаты
* **Внятные выводы** – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Небрежное его оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в тексте в явном виде.

Сдача отчетов осуществляется через систему AnyTask.


## Использование архитектуры SENNA для определения части речи

Домашнее задание написано по мотивам работы R. Collobert:

**Collobert, Ronan, Jason Weston, Léon Bottou, Michael Karlen, Koray Kavukcuoglu, and Pavel Kuksa. "Natural language processing (almost) from scratch." Journal of Machine Learning Research 12, no. Aug (2011): 2493-2537.**

В этом домашнем задании вам предстоит самостоятельно разработать архитектуру SENNA для определения части речи. 
SENNA – это простая архитектура нейронной сети, позволяющая достигнуть state-of-the-art результатов в нескольких задачах обработки текстов.  

Использование SENNA для определения части речи предполагает, что задача определения части речи для данного слова формулируется как задача классификации: пусть в размеченном корпусе всего $|T|$ (= tagset) различных тегов частей речи, тогда каждое слово $w$ относится к одному из $T$ классов. Для каждого слова из обучающих данных формируется собственный вектор признаков. Нейронная сеть обучается по всем векторам признаков для слов из обучающего множества. 

Подход к решению задачи классификации представлен в оригинальной статье на рис. 1 (Figure 1: Window approach network). Он состоит из следующих шагов (раздел 3.3.1):
1. Каждое слово представляется эмбеддингом: $w_i \rightarrow LT_{w^i}$, размерность эмбеддинга - $d$;
2. Для каждого слова формируется окно длины $k$ из $(k-1)/2$ соседних слов слева от данного слова  и $(k-1)/2$ соседних слов справа от данного слова, $k$ – нечетное. 
3. Для каждого слова формируется вектор признаков, состоящий из конкатенированных эмбеддингов слов из левого окна, данного слова и слов из правого окна. Итоговая размерность вектора признаков – $d \times k$. Именно этот вектор подается на вход нейронной сети;
4. Обучается нейронная сеть, имеющая один скрытый слой с $n_h$ нейроннами и нелинейной функцией активации $\theta$;
5. На выходном слое нейронной сети решается задача классификации на |T| классов, то есть, определяется часть речи для каждого слова. 

Если для слова невозможно найти $(k-1)/2$ соседних слов слева от данного слова  и $(k-1)/2$ соседних слов справа от данного слова – используется padding.


### Данные
1. Открытый корпус: https://github.com/dialogue-evaluation/morphoRuEval-2017/blob/master/OpenCorpora_Texts.rar
2. Предобученные эмбеддинги Facebook: https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ru.vec

### Часть 1 [2 балла] Подготовка данных
1. Прочитайте размеченные данные Открытого корпуса, используя nltk.corpus.reader.conll.ConllCorpusReader
2. Посчитайте количество предложений и число тегов частей речи;
3. Сформируйте тестовое и обучающее множество: первые 3/4 данных – обучающее множество;

Для каждого слова:
1. Определите его окно (слова слева и справа) размера $k$;
2. Сформируйте его вектор признаков.

In [266]:
from nltk.corpus.reader.conll import ConllCorpusReader
import numpy as np

In [2]:
reader = ConllCorpusReader('.', ['unamb_sent_14_6.conllu'], ('ignore', 'words', 'ignore', 'pos'))

Число предложений

In [260]:
sents = reader.tagged_sents()
N = len(sents)
print(N)

38508


Число тегов частей речи

In [261]:
all_pos = list(set(list(map(lambda x: x[1], reader.tagged_words()))))
NUM_CLASSES = len(all_pos)
print(NUM_CLASSES)

14


Делим предложения на обучающую и тестирующую часть

In [236]:
TRAIN = sents[:-N // 4]
TEST = sents[-N // 4:]

Функуции чтения эмбеддингов и подготовки датасета

In [237]:
def read_embeddings(max_words = -1):
    words = []
    embeddings = []
    file = open('wiki.ru.vec', 'r')
    file.readline()
    for i, line in enumerate(file):
        if max_words != -1 and i >= max_words:
            break
        line = line.strip()
        split_result = line.rsplit(maxsplit=300)
        word = split_result[0]
        embedding = np.array(split_result[1:],dtype=float)
        words.append(word)
        embeddings.append(embedding)
    return words, embeddings

In [238]:
def prepare_dataset(sents, word_to_idx, embeddings, pos_idx, k):
    p = (k - 1) // 2
    features = []
    labels = []
    for sent in sents:
        u = 0
        sent_embeddings = []
        unknown_embedding = np.zeros(300, dtype=float)
        sent_embeddings.extend([unknown_embedding] * p)
        for word, tag in sent:
            word = word.lower()
            if word in word_to_idx:
                word_embedding = embeddings[word_to_idx[word]]
            else:
                word_embedding = unknown_embedding
            sent_embeddings.append(word_embedding)
            labels.append(pos_idx[tag])
            u += 1
        sent_embeddings.extend([unknown_embedding] * p)
        for i in range(u):
            current = np.array(sent_embeddings[i:i + k])
            features.append(current.flatten())
    return np.array(features, dtype=float), np.array(labels,dtype=int)

In [239]:
def build_word_to_idx(words):
    res = {}
    for i, word in enumerate(words):
        res[word] = i
    return res

Считываем эмбеддинги (ограничиваемся 50000 самых популярных слов) и формируем датасет к окном ширины 3

In [225]:
words, embeddings = read_embeddings(50000)

In [240]:
word_to_idx = build_word_to_idx(words)
pos_idx = build_word_to_idx(all_pos)

In [241]:
k = 3
X_train, Y_train = prepare_dataset(TRAIN, word_to_idx, embeddings, pos_idx, k)
X_test, Y_test = prepare_dataset(TEST, word_to_idx, embeddings, pos_idx, k)

### Часть 2 [4 баллов] Архитектура нейронной сети

Архитектура нейронной сети состоит из следующих слов:
1. Входной слой: нейронная сеть получает на вход вектор признаков, состоящий из $k$ конкатенированных эмбеддингов;/
2. Скрытый слой: $n_h$ нейронов и нелинейная функция активации $\theta$;
3. Выходной слой:  $|T|$ нейронов для итоговой классификации.

Обучите нейронную сеть на обучающих данных.

In [254]:
import keras
from keras.layers import Input
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam

In [305]:
def build_model(n_h, k, act):
    model = Sequential()
    model.add(Dense(n_h, activation=act, input_shape=(k * 300,)))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

In [306]:
model = build_model(128, k, 'relu')

In [307]:
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [308]:
model.fit(X_train, Y_train, batch_size=64, epochs=1000, validation_split=0.1, \
          callbacks=[keras.callbacks.EarlyStopping('val_acc')])

Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 25s 79us/step - loss: 0.3827 - acc: 0.8721 - val_loss: 0.4079 - val_acc: 0.8576
Epoch 2/1000
315319/315319 [==============================] - 24s 76us/step - loss: 0.2860 - acc: 0.9003 - val_loss: 0.3814 - val_acc: 0.8679
Epoch 3/1000
315319/315319 [==============================] - 26s 82us/step - loss: 0.2583 - acc: 0.9093 - val_loss: 0.3755 - val_acc: 0.8691
Epoch 4/1000
315319/315319 [==============================] - 29s 91us/step - loss: 0.2406 - acc: 0.9149 - val_loss: 0.3825 - val_acc: 0.8716
Epoch 5/1000
315319/315319 [==============================] - 30s 94us/step - loss: 0.2267 - acc: 0.9190 - val_loss: 0.3854 - val_acc: 0.8722
Epoch 6/1000
315319/315319 [==============================] - 30s 94us/step - loss: 0.2154 - acc: 0.9226 - val_loss: 0.3964 - val_acc: 0.8730
Epoch 7/1000
315319/315319 [==============================] - 30s 96us/step - loss: 0.2065 - acc:

### Часть 3 [1 балл] Оценка качества

Протестируйте нейронную сеть на тестовых данных. Используйте accuracy для оценки качества модели.

In [309]:
_, accuracy = model.evaluate(X_test, Y_test)
print(accuracy)

107228/107228 [==============================] - 8s 70us/step
0.8908494049994847


Получили качество порядка 90%

### Часть 4 [1 балл] Оптимизация гиперпарметров

В эксперименте участвуют следующие гиперпараметры:
* $k$ – размер окна;
* $n_h$ – число нейронов на скрытом слое;
* $\theta$ – вид функции активации.

Оцените их влияние на качество модели. Как увеличение окна или числа нейронов влияет на итоговый показатель качества? Зависит ли итоговый показатель качества от функции активации на скрытом слое? 

In [316]:
def fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, thetha):
    print('Building model...')
    model = build_model(n_h, k, thetha)
    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train, Y_train, batch_size=64, epochs=1000, validation_split=0.1, \
          callbacks=[keras.callbacks.EarlyStopping('val_acc')])
    _, accuracy = model.evaluate(X_test, Y_test)
    return accuracy

In [317]:
def load_fit_and_measure_quality(k, n_h, thetha):
    print('Loading train dataset...')
    X_train, Y_train = prepare_dataset(TRAIN, word_to_idx, embeddings, pos_idx, k)
    print('Loading test dataset...')
    X_test, Y_test = prepare_dataset(TEST, word_to_idx, embeddings, pos_idx, k)
    return fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, thetha)

Оценим влияние ширины окна на показатель качества

In [278]:
for k in [1, 3, 5, 7]:
    print("Calculating for k=%d" % k)
    quality = load_fit_and_measure_quality(k, 128, 'relu')
    print("k=%d: Accuracy=%f" % (k, quality))

Calculating for k=1
Loading train dataset...
Loading test dataset...
Building model...
Epoch 1/4
350355/350355 [==============================] - 11s 31us/step - loss: 0.5107 - acc: 0.8205
Epoch 2/4
350355/350355 [==============================] - 10s 29us/step - loss: 0.4186 - acc: 0.8428
Epoch 3/4
350355/350355 [==============================] - 10s 29us/step - loss: 0.4065 - acc: 0.8458
Epoch 4/4
107228/107228 [==============================] - 6s 53us/step
k=1: Accuracy=0.841525
Calculating for k=3
Loading train dataset...
Loading test dataset...
Building model...
Epoch 1/4
350355/350355 [==============================] - 46s 132us/step - loss: 0.4089 - acc: 0.8641
Epoch 2/4
350355/350355 [==============================] - 21s 61us/step - loss: 0.3018 - acc: 0.8947
Epoch 3/4
350355/350355 [==============================] - 18s 52us/step - loss: 0.2751 - acc: 0.9032
Epoch 4/4
107228/107228 [==============================] - 7s 68us/step
k=3: Accuracy=0.894039
Calculating for k=5
Loa

Можно видеть, что использование окна шириной 3 заметно лучше чем единичное окно, однако дальнейшее увеличение не приводит к сильному росту качества. Хоть и есть некоторый прирост при k=5, он достаточно небольшой, практически в пределах погрешности. Поэтому в дальнейшем будем использовать модель с k=3

Подберем оптимальное значение нейронов скрытого слоя. Обычно для подобного параметра должен быть некоторый минимум, т.к. при небольшом количестве наблюдается недообучение, а при большом напротив - переобучение.

In [280]:
k = 3
X_train, Y_train = prepare_dataset(TRAIN, word_to_idx, embeddings, pos_idx, k)
X_test, Y_test = prepare_dataset(TEST, word_to_idx, embeddings, pos_idx, k)

In [318]:
for n_h in [8, 16, 32, 64, 128, 256]:
    print("Calculating for n_h=%d" % n_h)
    quality = fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, 'relu')
    print("n_h=%d: Accuracy=%f" % (n_h, quality))

Calculating for n_h=8
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 18s 58us/step - loss: 0.5683 - acc: 0.8234 - val_loss: 0.4958 - val_acc: 0.8368
Epoch 2/1000
315319/315319 [==============================] - 19s 61us/step - loss: 0.3723 - acc: 0.8763 - val_loss: 0.4664 - val_acc: 0.8450
Epoch 3/1000
315319/315319 [==============================] - 20s 62us/step - loss: 0.3532 - acc: 0.8815 - val_loss: 0.4586 - val_acc: 0.8479
Epoch 4/1000
315319/315319 [==============================] - 18s 57us/step - loss: 0.3440 - acc: 0.8845 - val_loss: 0.4498 - val_acc: 0.8506
Epoch 5/1000
315319/315319 [==============================] - 18s 58us/step - loss: 0.3379 - acc: 0.8858 - val_loss: 0.4400 - val_acc: 0.8523
Epoch 6/1000
315319/315319 [==============================] - 18s 58us/step - loss: 0.3333 - acc: 0.8872 - val_loss: 0.4417 - val_acc: 0.8528
Epoch 7/1000
315319/315319 [=============================

In [323]:
for n_h in [512, 1024, 2048]:
    print("Calculating for n_h=%d" % n_h)
    quality = fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, n_h, 'relu')
    print("n_h=%d: Accuracy=%f" % (n_h, quality))

Calculating for n_h=512
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 90s 286us/step - loss: 0.3572 - acc: 0.8787 - val_loss: 0.3856 - val_acc: 0.8659
Epoch 2/1000
315319/315319 [==============================] - 87s 275us/step - loss: 0.2674 - acc: 0.9067 - val_loss: 0.3716 - val_acc: 0.8709
Epoch 3/1000
315319/315319 [==============================] - 86s 273us/step - loss: 0.2373 - acc: 0.9159 - val_loss: 0.3772 - val_acc: 0.8713
Epoch 4/1000
107228/107228 [==============================] - 13s 126us/step
n_h=512: Accuracy=0.891194
Calculating for n_h=1024
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
 96832/315319 [========>.....................] - ETA: 2:00 - loss: 0.4300 - acc: 0.8568

KeyboardInterrupt: 

Выберем функцию активации

In [322]:
for nonlinearity in ['relu', 'elu', 'selu', 'sigmoid', 'tanh']:
    print("Calculating for %s" % nonlinearity)
    quality = fit_and_measure_quality(X_train, Y_train, X_test, Y_test, k, 256, nonlinearity)
    print("function=%s: Accuracy=%f" % (nonlinearity, quality))

Calculating for relu
Building model...
Train on 315319 samples, validate on 35036 samples
Epoch 1/1000
315319/315319 [==============================] - 48s 152us/step - loss: 0.3680 - acc: 0.8761 - val_loss: 0.4004 - val_acc: 0.8620
Epoch 2/1000
315319/315319 [==============================] - 55s 175us/step - loss: 0.2751 - acc: 0.9036 - val_loss: 0.3809 - val_acc: 0.8689
Epoch 3/1000
 19712/315319 [>.............................] - ETA: 52s - loss: 0.2387 - acc: 0.9160

KeyboardInterrupt: 

### Часть 5 [2 балла] Анализ ошибок
1. Привидите примеры из тестового множества, на которых нейронная сеть ошибается. Объясните, почему возникают ошибки.
2. Протестируйте нейронную сеть на произвольном предложении (не из тестовых данных). Возникают ли ошибки? Почему?